In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import pickle
import plotly.express as px
import statsmodels.formula.api as smf
import plotly.graph_objects as go
from scripts.python.routines.manifest import get_manifest
from scripts.python.routines.plot.save import save_figure
from scripts.python.routines.plot.layout import add_layout
from statsmodels.stats.multitest import multipletests
import plotly.io as pio
pio.kaleido.scope.mathjax = None
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from scipy.stats import mannwhitneyu, median_test
import matplotlib.pyplot as plt
import pathlib
from tqdm import tqdm
from src.utils.plot.bioinfokit import mhat, volcano
import gseapy as gp
import mygene
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA, TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning, FastICA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
import upsetplot as upset
import missingno as msno
from pyod.models.lunar import LUNAR
from matplotlib_venn import venn2, venn2_circles

# Init dnam data

In [ ]:
dataset = "GSEUNN"
path = f"E:/YandexDisk/Work/pydnameth/datasets"
datasets_info = pd.read_excel(f"{path}/datasets.xlsx", index_col='dataset')
platform = datasets_info.loc[dataset, 'platform']
manifest = get_manifest(platform, path=path)
manifest['CHR'] = manifest['chr'].str[3::]

dnam_suffix = "_harm"

path_save = f"{path}/{platform}/{dataset}/special/040_report_mega_2022"
pathlib.Path(f"{path_save}").mkdir(parents=True, exist_ok=True)

pheno = pd.read_excel(f"{path}/{platform}/{dataset}/pheno.xlsx", index_col="index")
pheno.index.name = "index"
pheno.drop("I64_old", inplace=True)
betas = pd.read_pickle(f"{path}/{platform}/{dataset}/betas{dnam_suffix}.pkl")
dnam_feats = betas.columns.values
df_dnam = pd.merge(pheno, betas, left_index=True, right_index=True)
df_dnam = df_dnam.loc[(df_dnam["Status"] == "Control") & (df_dnam["Region"] == "Central"), :]

print(f"df_dnam shape: {df_dnam.shape}")

## Participants figure

In [ ]:
path_local = "part_1.2_sex_specific_dnam"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)

# Params for figure
binrange = [10, 105]
bins = 15
df_fig = df_dnam.loc[:, ["Age", "Sex"]]
num_F = df_fig.loc[df_fig['Sex'] == 'F', :].shape[0]
num_M = df_fig.loc[df_fig['Sex'] == 'M', :].shape[0]
df_fig["Sex"].replace({"F": f"F ({num_F})", "M": f"M ({num_M})"}, inplace=True)

sns.set_theme(style='whitegrid')
sns.histplot(
    data=df_fig,
    hue_order=[f"F ({num_F})", f"M ({num_M})"],
    binrange=binrange,
    bins=bins,
    x="Age",
    hue="Sex",
    palette={f"F ({num_F})": "red", f"M ({num_M})": "blue"},
)
plt.savefig(f"{path_save}/{path_local}/hist_Sex.png", bbox_inches='tight')
plt.savefig(f"{path_save}/{path_local}/hist_Sex.pdf", bbox_inches='tight')
plt.clf()

# Aux DNAm data

In [ ]:
problem = {
    "Color": {
        "F": "red",
        "M": "blue",
    },
    "Filter": {
        "F": df_dnam["Sex"] == "F",
        "M": df_dnam["Sex"] == "M",
    },
    "BaseFilter": (df_dnam["Sex"] == "F"),
    "BasePart": "F"
}

# Create data for R

In [ ]:
pathlib.Path(f"{path_save}/data_for_R").mkdir(parents=True, exist_ok=True)

betas_R = df_dnam.loc[:, dnam_feats]
betas_R = betas_R.T
betas_R.index.name = "CpG"
betas_R.to_pickle(f"{path_save}/data_for_R/betas.pkl")

pheno_R = df_dnam.loc[:, ["Age", "Sex", "Sentrix_ID", "Sentrix_Position"]]
pheno_R.to_pickle(f"{path_save}/data_for_R/pheno.pkl")

# DNAm features

In [ ]:
path_local = "part_1.2_sex_specific_dnam/age"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)

In [ ]:
df_dnam_age = pd.read_csv(f"{path_save}/data_for_R/DMP_age.csv", index_col="CpG")
df_dnam_age["CpG"] = df_dnam_age.index.values
df_dnam_age['print'] = df_dnam_age.apply(lambda row: f"{row['CpG']} ({row['gene']})", axis=1)
top_to_hightlight = df_dnam_age["print"].values[0:5]
df_dnam_age['log_pval'] = -np.log10(df_dnam_age["adj.P.Val"])
sns.set_theme(style='whitegrid')
df_dnam_age.sort_values(["MAPINFO"], ascending=[True], inplace=True)
mhat(
    df=df_dnam_age,
    chr='CHR',
    pv='adj.P.Val',
    path=f"{path_save}/{path_local}",
    valpha=1,
    markernames=tuple(top_to_hightlight),
    markeridcol='print',
    gstyle=2,
    dim=(12,4),
    axtickfontsize=8
)

# DNAm analysis

## Cells

In [ ]:
path_local = "part_1.2_sex_specific_dnam/cells"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)
cells = ["CD8T", "CD4T", "NK", "Bcell", "Mono", "Gran"]
cells = [f"{x}{dnam_suffix}" for x in cells]
df_cells = pd.DataFrame()
for cell in tqdm(cells):
    vals = {}
    for group in problem["Filter"]:

        vals[group] = df_dnam.loc[problem["Filter"][group], cell].values
        df_cells.at[cell, f"mean_{group}"] = np.mean(vals[group])
        df_cells.at[cell, f"median_{group}"] = np.median(vals[group])
        df_cells.at[cell, f"q75_{group}"], df_cells.at[cell, f"q25_{group}"] = np.percentile(vals[group], [75 ,25])
        df_cells.at[cell, f"iqr_{group}"] = df_cells.at[cell, f"q75_{group}"] - df_cells.at[cell, f"q25_{group}"]

    _, pval = mannwhitneyu(*vals.values(), alternative='two-sided')
    df_cells.at[cell, "pval"] = pval

_, df_cells["pval_fdr_bh"], _, _ = multipletests(df_cells["pval"], 0.05, method='fdr_bh')
df_cells.to_excel(f"{path_save}/{path_local}/cells.xlsx", index=True)

In [ ]:

for cell in tqdm(cells):

    vals = {}
    for group in problem["Filter"]:
        vals[group] = df_dnam.loc[problem["Filter"][group], cell].values
        print(f"{group}: {len(vals[group])}")

    dist_num_bins = 20
    fig = go.Figure()
    for group in problem["Filter"]:
        fig.add_trace(
            go.Violin(
                y=vals[group],
                name=group,
                box_visible=True,
                meanline_visible=True,
                showlegend=False,
                line_color='black',
                fillcolor=problem["Color"][group],
                marker = dict(color=problem["Color"][group], line=dict(color='black',width=0.3), opacity=0.8),
                points='all',
                bandwidth = np.ptp(vals[group]) / dist_num_bins,
                opacity=0.8
            )
        )
    add_layout(fig, "", f"{cell}", f"p-value: {df_cells.at[cell, 'pval_fdr_bh']:0.2e}")
    fig.update_layout(title_xref='paper')
    fig.update_layout(legend_font_size=20)
    fig.update_layout(legend={'itemsizing': 'constant'})
    fig.update_layout(
        violingap=0.35,
        violingroupgap=0.35,
        width=850,
        height=600,
        margin=go.layout.Margin(
            l=150,
            r=50,
            b=75,
            t=100,
            pad=0,
        )
    )
    fig.update_layout(legend_y=1.01)
    save_figure(fig, f"{path_save}/{path_local}/{cell}")

## Age Accelerations

In [ ]:
path_local = "part_1.2_sex_specific_dnam/age_accelerations"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)
age_types = ['DNAmAgeHannum', 'DNAmAge', 'DNAmPhenoAge', 'DNAmGrimAge']
age_types = [f"{x}{dnam_suffix}" for x in age_types]
df_aas = pd.DataFrame(index=[f"{x}Acc" for x in age_types], columns=["pval", "pval_fdr_bh"])
for age_type in tqdm(age_types):
    formula = f"{age_type} ~ Age"
    model = smf.ols(formula=formula, data=df_dnam.loc[problem["BaseFilter"]]).fit()
    df_dnam[f"{age_type}_linear_pred"] = model.predict(df_dnam)
    y_pred = model.predict(pheno)
    df_dnam[f"{age_type}Acc"] = df_dnam[age_type] - df_dnam[f"{age_type}_linear_pred"]

    vals = {}
    for group in problem["Filter"]:

        vals[group] = df_dnam.loc[problem["Filter"][group], f"{age_type}Acc"].values
        df_aas.at[f"{age_type}Acc", f"mean_{group}"] = np.mean(vals[group])
        df_aas.at[f"{age_type}Acc", f"median_{group}"] = np.median(vals[group])
        df_aas.at[f"{age_type}Acc", f"q75_{group}"], df_aas.at[f"{age_type}Acc", f"q25_{group}"] = np.percentile(vals[group], [75 ,25])
        df_aas.at[f"{age_type}Acc", f"iqr_{group}"] = df_aas.at[f"{age_type}Acc", f"q75_{group}"] - df_aas.at[f"{age_type}Acc", f"q25_{group}"]
        print(f"{group}: {len(vals[group])}")

    _, pval = mannwhitneyu(*vals.values(), alternative='two-sided')
    df_aas.at[f"{age_type}Acc", "pval"] = pval

_, df_aas["pval_fdr_bh"], _, _ = multipletests(df_aas["pval"], 0.05, method='fdr_bh')
df_aas.to_excel(f"{path_save}/{path_local}/aas.xlsx", index=True)

In [ ]:
for age_type in tqdm(age_types):

    vals = {}
    for group in problem["Filter"]:
        vals[group] = df_dnam.loc[problem["Filter"][group], f"{age_type}Acc"].values
        print(f"{group}: {len(vals[group])}")

    # Plot without longevity =======================================================
    dist_num_bins = 20
    fig = go.Figure()
    for group in problem["Filter"]:
        fig.add_trace(
            go.Violin(
                y=vals[group],
                name=group,
                box_visible=True,
                meanline_visible=True,
                showlegend=False,
                line_color='black',
                fillcolor=problem["Color"][group],
                marker=dict(color=problem["Color"][group], line=dict(color='black',width=0.3), opacity=0.8),
                points='all',
                bandwidth=np.ptp(vals[group]) / dist_num_bins,
                opacity=0.8,
            )
        )
    add_layout(fig, "", f"{age_type}Acc", f"p-value: {df_aas.at[f'{age_type}Acc', 'pval_fdr_bh']:0.2e}")
    fig.update_layout(title_xref='paper')
    fig.update_layout(legend_font_size=20)
    fig.update_layout(legend= {'itemsizing': 'constant'})
    fig.update_layout(
        violingap=0.35,
        violingroupgap=0.35,
        width=850,
        height=600,
        margin=go.layout.Margin(
            l=150,
            r=50,
            b=75,
            t=100,
            pad=0,
        )
    )
    fig.update_layout(legend_y=1.01)
    save_figure(fig, f"{path_save}/{path_local}/violin_{age_type}Acc")

    min_val = df_dnam[["Age", age_type]].min().min()
    max_val = df_dnam[["Age", age_type]].max().max()
    shift_val = max_val - min_val
    min_val -= 0.05 * shift_val
    max_val += 0.05 * shift_val

    # Plot without longevity =======================================================
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=[min_val, max_val],
            y=[min_val, max_val],
            showlegend=False,
            name="",
            mode="lines",
            marker_color="black",
            marker=dict(
                size=8,
                opacity=0.75,
                line=dict(
                    color="black",
                    width=0.5
                )
            )
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df_dnam.loc[problem["BaseFilter"], f"Age"].values,
            y=df_dnam.loc[problem["BaseFilter"], f"{age_type}_linear_pred"].values,
            showlegend=False,
            name="",
            mode="lines",
            line=dict(width=5),
            marker_color=problem["Color"][problem["BasePart"]],
            marker=dict(
                size=8,
                opacity=0.75,
                line=dict(
                    color="black",
                    width=0.5
                )
            )
        )
    )
    for group in problem["Filter"]:
        fig.add_trace(
            go.Scatter(
                x=df_dnam.loc[problem["Filter"][group], f"Age"].values,
                y=df_dnam.loc[problem["Filter"][group], f"{age_type}"].values,
                showlegend=True,
                name=group,
                mode="markers",
                line_color=problem["Color"][group],
                marker=dict(
                    size=8,
                    opacity=0.75,
                    line=dict(
                        color="black",
                        width=0.5
                    )
                )
            )
        )
    add_layout(fig, f"Age", f"{age_type}", f"")
    fig.update_layout(legend_font_size=20)
    fig.update_layout(legend= {'itemsizing': 'constant'})
    fig.update_xaxes(autorange=False)
    fig.update_yaxes(autorange=False)
    fig.update_layout(title_xref='paper')
    fig.update_layout(xaxis_range=[min_val, max_val])
    fig.update_layout(yaxis_range=[min_val, max_val])
    fig.update_layout(
        width=650,
        height=600,
        margin=go.layout.Margin(
            l=100,
            r=50,
            b=100,
            t=50,
            pad=0,
        )
    )
    save_figure(fig, f"{path_save}/{path_local}/scatter_{age_type}")

## ChAMP DMPs

### Setup

In [ ]:
pval_lim = 0.001
fc_lim = 0.05
path_local = f"part_1.2_sex_specific_dnam/DMPs_Sex_pval({pval_lim:0.2e})_fc({fc_lim:0.2e})"
pathlib.Path(f"{path_save}/{path_local}").mkdir(parents=True, exist_ok=True)

### Read ChAMP results

In [ ]:
df_dmps = pd.read_csv(f"{path_save}/data_for_R/DMP_sex.csv", index_col="CpG")
df_dmps["CpG"] = df_dmps.index.values
df_dmps.sort_values(["adj.P.Val"], ascending=[True], inplace=True)
df_dmps['print'] = df_dmps.apply(lambda row: f"{row['CpG']} ({row['gene']})", axis=1)
df_dmps['log_pval'] = -np.log10(df_dmps["adj.P.Val"])

### Obtain gene list

In [ ]:
df_dmps_selected = df_dmps.loc[(df_dmps["adj.P.Val"] < pval_lim) & ((df_dmps["logFC"] < -fc_lim) | (df_dmps["logFC"] > fc_lim)), :]
df_dmps_selected.sort_values(["adj.P.Val"], ascending=[True], inplace=True)
top_to_hightlight = df_dmps_selected["print"].values[0:1]
df_dmps_selected.to_excel(f"{path_save}/{path_local}/selected.xlsx")
genes_dmps_selected = set()
for cpg in df_dmps_selected.index.values:
    genes_raw = manifest.at[cpg, 'Gene']
    if isinstance(genes_raw, str):
        genes = genes_raw.split(';')
        genes_dmps_selected.update(set(genes))
if 'non-genic' in genes_dmps_selected:
    genes_dmps_selected.remove('non-genic')
if ' ' in genes_dmps_selected:
    genes_dmps_selected.remove(' ')
genes_dmps_selected = list(genes_dmps_selected)
genes_dmps_df = pd.DataFrame({'gene':genes_dmps_selected})
genes_dmps_df.to_excel(f"{path_save}/{path_local}/genes.xlsx", index=False)
print(f"Number of CpGs: {df_dmps_selected.shape[0]}")
print(f"Number of genes: {genes_dmps_df.shape[0]}")

In [ ]:
sns.set_theme(style='whitegrid')
df_dmps.sort_values(["MAPINFO"], ascending=[True], inplace=True)
mhat(
    df=df_dmps,
    chr='CHR',
    pv='adj.P.Val',
    path=f"{path_save}/{path_local}",
    valpha=1,
    markernames=tuple(top_to_hightlight),
    markeridcol='print',
    gstyle=2,
    dim=(12, 4),
    axtickfontsize=8
)
sns.set_theme(style='whitegrid')
volcano(
    df=df_dmps,
    lfc='logFC',
    pv='adj.P.Val',
    pv_thr=(pval_lim, pval_lim),
    lfc_thr=(fc_lim, fc_lim),
    path=f"{path_save}/{path_local}",
    genenames=tuple(top_to_hightlight),
    geneid='print',
    gstyle=2,
    sign_line=True,
    color=("limegreen", "grey", "royalblue")
)

### Perform dimensionality reduction

In [ ]:
feats_dim_red = df_dmps_selected["CpG"].values
df_dnam_dim_red = df_dnam.loc[:, list(feats_dim_red) + ["Age", "Sex", "Region"]]
data_dim_red = df_dnam_dim_red.loc[:, feats_dim_red].values
classes_dim_red = df_dnam_dim_red.loc[:, 'Sex'].values

In [ ]:
print(f"PCA")
pca = PCA(n_components=2, whiten=False)
data_pca = pca.fit_transform(data_dim_red)
df_dnam_dim_red['PC 1'] = data_pca[:, 0]
df_dnam_dim_red['PC 2'] = data_pca[:, 1]

print(f"Incremental PCA")
n_batches = 32
ipca = IncrementalPCA(n_components=2)
for data_batch in np.array_split(data_dim_red, n_batches):
    ipca.partial_fit(data_batch)
data_ipca = ipca.transform(data_dim_red)
df_dnam_dim_red['Incremental PC 1'] = data_ipca[:, 0]
df_dnam_dim_red['Incremental PC 2'] = data_ipca[:, 1]

print(f"Kernel PCA")
kpca = KernelPCA(kernel='rbf', fit_inverse_transform=True, gamma=None, n_components=2)
data_kpca = kpca.fit_transform(data_dim_red)
df_dnam_dim_red['Kernel PC 1'] = data_kpca[:, 0]
df_dnam_dim_red['Kernel PC 2'] = data_kpca[:, 1]

print(f"SVD")
tsvd = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=5)
tsvd.fit(data_dim_red)
data_svd = tsvd.transform(data_dim_red)
df_dnam_dim_red['SVD 1'] = data_svd[:, 0]
df_dnam_dim_red['SVD 2'] = data_svd[:, 1]

print(f"GRP")
GRP = GaussianRandomProjection(n_components=2, eps=0.5)
GRP.fit(data_dim_red)
data_grp = GRP.transform(data_dim_red)
df_dnam_dim_red['Gaussian Random Projection 1'] = data_grp[:, 0]
df_dnam_dim_red['Gaussian Random Projection 2'] = data_grp[:, 1]

print(f"SRP")
SRP = SparseRandomProjection(n_components=2, density='auto', eps=0.5, dense_output=False)
SRP.fit(data_dim_red)
data_srp = SRP.transform(data_dim_red)
df_dnam_dim_red['Sparse Random Projection 1'] = data_srp[:, 0]
df_dnam_dim_red['Sparse Random Projection 2'] = data_srp[:, 1]

print(f"MDS")
mds = MDS(n_components=2, metric=True)
data_mds = mds.fit_transform(data_dim_red)
df_dnam_dim_red['Multi Dimensional Scale 1'] = data_mds[:, 0]
df_dnam_dim_red['Multi Dimensional Scale 2'] = data_mds[:, 1]

print(f"ISOMAP")
isomap = Isomap(n_components=2, n_neighbors=5)
isomap.fit(data_dim_red)
data_isomap = isomap.transform(data_dim_red)
df_dnam_dim_red['IsoMap 1'] = data_isomap[:, 0]
df_dnam_dim_red['IsoMap 2'] = data_isomap[:, 1]

print(f"MiniBatchDictionaryLearning")
miniBatchDictLearning = MiniBatchDictionaryLearning(n_components=2, batch_size=200, alpha=1, n_iter=25)
miniBatchDictLearning.fit(data_dim_red)
data_batch = miniBatchDictLearning.fit_transform(data_dim_red)
df_dnam_dim_red['MBDL 1'] = data_batch[:, 0]
df_dnam_dim_red['MBDL 2'] = data_batch[:, 1]

print(f"ICA")
fastICA = FastICA(n_components=2, algorithm='parallel', whiten=True, tol=1e-3, max_iter=1000)
data_ica = fastICA.fit_transform(data_dim_red)
df_dnam_dim_red['IC 1'] = data_ica[:, 0]
df_dnam_dim_red['IC 2'] = data_ica[:, 1]

print(f"t-SNE")
tsne = TSNE(n_components=2, learning_rate=300, perplexity=30, early_exaggeration=12, init='random')
data_tsne = tsne.fit_transform(data_dim_red)
df_dnam_dim_red['tSNE 1'] = data_tsne[:, 0]
df_dnam_dim_red['tSNE 2'] = data_tsne[:, 1]

print(f"LLE")
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, method='modified')
lle.fit(data_dim_red)
data_lle = lle.transform(data_dim_red)
df_dnam_dim_red['LLE 1'] = data_lle[:, 0]
df_dnam_dim_red['LLE 2'] = data_lle[:, 1]

In [ ]:
from itertools import chain

pathlib.Path(f"{path_save}/{path_local}/dim_red").mkdir(parents=True, exist_ok=True)
dim_red_methods_dict = {
    'PCA': ['PC 1', 'PC 2'],
    'IncrementalPCA': ['Incremental PC 1', 'Incremental PC 2'],
    'KernelPCA': ['Kernel PC 1', 'Kernel PC 2'],
    'SingularValueDecomposition': ['SVD 1', 'SVD 2'],
    'GaussianRandomProjection': ['Gaussian Random Projection 1', 'Gaussian Random Projection 2'],
    'SparseRandomProjection': ['Sparse Random Projection 1', 'Sparse Random Projection 2'],
    'MultiDimensionalScaling': ['Multi Dimensional Scale 1', 'Multi Dimensional Scale 2'],
    'Isomap': ['IsoMap 1', 'IsoMap 2'],
    'MiniBatchDictionaryLearning': ['MBDL 1', 'MBDL 2'],
    'ICA': ['IC 1', 'IC 2'],
    'T-SNE': ['tSNE 1', 'tSNE 2'],
    'LocallyLinearEmbedding': ['LLE 1', 'LLE 2']
}
df_dnam_dim_red.loc[:, list(chain(*dim_red_methods_dict.values()))].to_excel(f"{path_save}/{path_local}/dim_red/table.xlsx", index=True)
for method in dim_red_methods_dict:
    x_col = dim_red_methods_dict[method][0]
    y_col = dim_red_methods_dict[method][1]

    # Plot without longevity =======================================================
    fig = go.Figure()
    for group in problem["Color"]:
        fig.add_trace(
            go.Scatter(
                x=df_dnam_dim_red.loc[problem["Filter"][group], x_col].values,
                y=df_dnam_dim_red.loc[problem["Filter"][group], y_col].values,
                showlegend=True,
                name=group,
                mode="markers",
                line_color=problem["Color"][group],
                marker=dict(
                    size=8,
                    opacity=0.8,
                    color=problem["Color"][group],
                    symbol="circle",
                    line=dict(
                        color="black",
                        width=1
                    )
                )
            )
        )
    add_layout(fig, x_col, y_col, f"")
    fig.update_layout(legend_font_size=20)
    fig.update_layout(legend= {'itemsizing': 'constant'})
    fig.update_layout(
        width=650,
        height=600,
        margin=go.layout.Margin(
            l=100,
            r=50,
            b=100,
            t=50,
            pad=0,
        )
    )
    save_figure(fig, f"{path_save}/{path_local}/dim_red/{method}")

### Perform GSEA for selected gene libraries

In [ ]:
pathlib.Path(f"{path_save}/{path_local}/GSEA").mkdir(parents=True, exist_ok=True)
libraries = gp.get_library_name("Human")
df_libraries = pd.DataFrame(index=libraries)
df_libraries.to_excel(f"{path_save}/GSEA_libs/libraries.xlsx", index=True)

genes_dict_of_lists = {
    "origin": genes_dmps_selected,
}

for genes in genes_dict_of_lists:
    dfs_enrichr = []
    for genes_list in libraries:
        pathlib.Path(f"{path_save}/{path_local}/GSEA/{genes}/{genes_list}").mkdir(parents=True, exist_ok=True)
        df_enrichr = gp.enrichr(
            gene_list=genes_dict_of_lists[genes],
            gene_sets=genes_list,
            organism='Human',
            outdir=f"{path_save}/{path_local}/GSEA/{genes}/{genes_list}",
            cutoff=1.00,
            verbose=True,
            no_plot=True
        )
        dfs_enrichr.append(df_enrichr.results)
    dfs_enrichr = pd.concat(dfs_enrichr)
    dfs_enrichr.to_excel(f"{path_save}/{path_local}/GSEA/{genes}/results.xlsx", index=True)
    dfs_enrichr.to_pickle(f"{path_save}/{path_local}/GSEA/{genes}/results.pkl")

### Plot significant GSEA terms

In [ ]:
libraries_file = [
    "libraries_target_GO_Biological_Process",
    "libraries_target_GO_Cellular_Component",
    "libraries_target_GO_Molecular_Function",
    "libraries_target_nonGO",
]

for library_file in libraries_file:
    libraries_target = pd.read_excel(f"{path_save}/GSEA_libs/{library_file}.xlsx")["library"].values

    gsea_cols = ["Gene_set", "Term", "Overlap", "P-value", "Adjusted P-value", "Odds Ratio", "Combined Score"]
    for genes in ["origin"]:

        dfs_enrichr = pd.read_pickle(f"{path_save}/{path_local}/GSEA/{genes}/results.pkl")
        dfs_enrichr = dfs_enrichr.loc[(dfs_enrichr["Adjusted P-value"] < 0.05) & (dfs_enrichr["Gene_set"].isin(libraries_target)), gsea_cols]
        dfs_enrichr.index = range(len(dfs_enrichr))

        if dfs_enrichr.empty == False:
            dfs_enrichr[r'$ -\log_{10}(\mathrm{p-value})$'] = -np.log10(dfs_enrichr.loc[:, 'Adjusted P-value'].values)
            dfs_enrichr.rename(columns={'Gene_set': 'Gene Library'}, inplace=True)
            dfs_enrichr.to_excel(f"{path_save}/{path_local}/GSEA/{genes}/terms_{library_file}.xlsx")
            plt.figure(figsize=(10, 0.5 * dfs_enrichr.shape[0]))
            sns.set_theme(style='whitegrid', font_scale=2)
            bar = sns.barplot(
                data=dfs_enrichr,
                hue="Gene Library",
                y=dfs_enrichr.index,
                x=r'$ -\log_{10}(\mathrm{p-value})$',
                palette=list(px.colors.qualitative.Alphabet) + list(px.colors.qualitative.Dark24) + list(px.colors.qualitative.Light24),
                edgecolor='black',
                orient="h",
                dodge=False
            )
            bar.set_yticklabels(dfs_enrichr["Term"])
            sns.move_legend(bar, "upper left", bbox_to_anchor=(1, 1))
            plt.savefig(f"{path_save}/{path_local}/GSEA/{genes}/terms_{library_file}.png", bbox_inches='tight')
            plt.savefig(f"{path_save}/{path_local}/GSEA/{genes}/terms_{library_file}.pdf", bbox_inches='tight')
            plt.close()

### Plot upset plots for target terms

In [ ]:
df_ss_2014 = pd.read_excel(f"{path_save}/part_1.2_sex_specific_dnam/12864_2014_6710_MOESM4_ESM.xlsx", index_col="Target ID")
cpgs_ss_2014 = df_ss_2014.index.values

df_ss_2022 = pd.read_csv(f"{path_save}/part_1.2_sex_specific_dnam/13148_2022_1279_MOESM1_ESM.csv", index_col="Row.names")
cpgs_ss_2022 = df_ss_2022.index.values

cpgs_ss_unn = df_dmps_selected.index.values

cpgs_lists = {
    'McCarthy2014': cpgs_ss_2014,
    'Grant2022': cpgs_ss_2022,
    'UNN': cpgs_ss_unn
}
all_cpgs = results_union = set().union(*list(cpgs_lists.values()))
df_upset = pd.DataFrame(index=all_cpgs)
for k, v in cpgs_lists.items():
    df_upset[k] = df_upset.index.isin(v)
df_upset = df_upset.set_index(list(cpgs_lists.keys()))
tmp = plt.figure(figsize=(14, 8))
upset_fig = upset.UpSet(df_upset, subset_size='count', show_counts=True, element_size=None, min_degree=1).plot(tmp)
plt.savefig(f"{path_save}/part_1.2_sex_specific_dnam/upset.png", bbox_inches='tight')
plt.savefig(f"{path_save}/part_1.2_sex_specific_dnam/upset.pdf", bbox_inches='tight')

### Plot region enrichment

In [ ]:
pathlib.Path(f"{path_save}/{path_local}/region_enrichment").mkdir(parents=True, exist_ok=True)
pval_show_type = "color" # "cross"
orders = {
    'CHR': [str(x) for x in range(1, 24)],
    'RELATION_TO_UCSC_CPG_ISLAND': ['S_Shelf', 'S_Shore', 'Island', 'N_Shore', 'N_Shelf', 'OpenSea'],
    'UCSC_REFGENE_GROUP': ['TSS1500', 'TSS200', '5\'UTR', '1stExon', 'Body', '3\'UTR']
}
col_names = {
    'CHR': "CHR",
    'RELATION_TO_UCSC_CPG_ISLAND': "Relation_to_Island",
    'UCSC_REFGENE_GROUP': "UCSC_RefGene_Group"
}
fig_sizes = {
    'CHR': (17, 10),
    'RELATION_TO_UCSC_CPG_ISLAND': (5, 10),
    'UCSC_REFGENE_GROUP': (5, 10)
}
colors = {
    'CHR': px.colors.qualitative.Dark24,
    'RELATION_TO_UCSC_CPG_ISLAND': px.colors.qualitative.Light24[17:23],
    'UCSC_REFGENE_GROUP': px.colors.qualitative.Light24[11:17]
}
df_dmps_fisher_target = manifest.loc[df_dmps_selected.index.values, :]
df_dmps_fisher_global = manifest.loc[df_dmps.index.values, :]
df_dmps_fisher_padding = df_dmps_fisher_global.loc[~df_dmps_fisher_global.index.isin(df_dmps_selected.index.values), :]
for var in orders:
    columns=["11", "12", "21", "22", "sum", "pval", "odds_ratio"]
    df_var = pd.DataFrame(index=orders[var], columns=columns, data=np.zeros((len(orders[var]), len(columns))))
    df_var.index.name = col_names[var].replace("_", " ")
    for var_val in orders[var]:
        contingency_table = pd.DataFrame(index=["specific", "non-specific"], columns=["in_val", "not_in_val"])
        contingency_table.at["specific", "in_val"] = df_dmps_fisher_target.loc[df_dmps_fisher_target[col_names[var]] == var_val, :].shape[0]
        contingency_table.at["specific", "not_in_val"] = df_dmps_fisher_target.loc[df_dmps_fisher_target[col_names[var]] != var_val, :].shape[0]
        contingency_table.at["non-specific", "in_val"] = df_dmps_fisher_padding.loc[df_dmps_fisher_padding[col_names[var]] == var_val, :].shape[0]
        contingency_table.at["non-specific", "not_in_val"] = df_dmps_fisher_padding.loc[df_dmps_fisher_padding[col_names[var]] != var_val, :].shape[0]
        df_var.at[var_val, "11"] = contingency_table.at["specific", "in_val"]
        df_var.at[var_val, "12"] = contingency_table.at["specific", "not_in_val"]
        df_var.at[var_val, "21"] = contingency_table.at["non-specific", "in_val"]
        df_var.at[var_val, "22"] = contingency_table.at["non-specific", "not_in_val"]
        df_var.at[var_val, "sum"] = contingency_table.values.sum()
        odds_ratio, pval = stats.fisher_exact(contingency_table.to_numpy(), alternative='two-sided')
        if np.isnan(odds_ratio):
            odds_ratio = 1.0
        df_var.at[var_val, "odds_ratio"], df_var.at[var_val, "pval"] = odds_ratio, pval
    _, df_var['pval_fdr_bh'], _, _ = multipletests(df_var['pval'].values, 0.05, method='fdr_bh')
    df_var[r'$ \log_{10}(\mathrm{Odds\ ratio})$'] = np.log10(df_var.loc[:, 'odds_ratio'].values)
    df_var[r'$ -\log_{10}(\mathrm{p-value})$'] = -np.log10(df_var.loc[:, 'pval_fdr_bh'].values)
    df_var.to_excel(f"{path_save}/{path_local}/region_enrichment/fisher_{var}.xlsx")

    plt.figure(figsize=fig_sizes[var])
    plt.xticks(rotation=90)
    sns.set_theme(style='whitegrid', font_scale=2)
    if pval_show_type == "color":
        plot = plt.scatter(df_var.index, df_var.loc[:, r'$ \log_{10}(\mathrm{Odds\ ratio})$'].values, c=df_var.loc[:, r'$ -\log_{10}(\mathrm{p-value})$'].values, cmap='Reds')
        plt.clf()
        cbar = plt.colorbar(plot)
        plt.xticks(rotation=90)
        cbar.set_label(r"$-\log_{10}(\mathrm{p-value})$", horizontalalignment='center')
        ax = sns.barplot(data=df_var, x=df_var.index, y=r'$ \log_{10}(\mathrm{Odds\ ratio})$', hue=r'$ -\log_{10}(\mathrm{p-value})$', palette='Reds', dodge=False, edgecolor='black')
        ax.legend_.remove()
    else:
        bar = sns.barplot(data=df_var, x=df_var.index, y=r'$ \log_{10}(\mathrm{Odds\ ratio})$', palette=colors[var], edgecolor='black')
        for bar_index, this_bar in enumerate(bar.patches):
            if df_var.at[df_var.index[bar_index], "pval_fdr_bh"] < 0.05:
                this_bar.set_hatch('x')
            this_bar.set_edgecolor('skyblue')
    plt.savefig(f"{path_save}/{path_local}/region_enrichment/fisher_{var}.png", bbox_inches='tight')
    plt.savefig(f"{path_save}/{path_local}/region_enrichment/fisher_{var}.pdf", bbox_inches='tight')
    plt.close()

### Plot example

In [ ]:
n_top = 10
dist_num_bins = 25
pathlib.Path(f"{path_save}/{path_local}/examples").mkdir(parents=True, exist_ok=True)
df_dmps_top = df_dmps.sort_values(['adj.P.Val'], ascending=[True]).head(n_top)
for cpg_id, (cpg, row) in enumerate(df_dmps_top.iterrows()):
    pval = row['adj.P.Val']
    log_fc = row['logFC']
    gene = manifest.at[cpg, 'Gene']

    fig = go.Figure()
    for group in problem["Filter"]:
        vals = df_dnam.loc[problem["Filter"][group], cpg].values
        fig.add_trace(
            go.Violin(
                y=vals,
                name=group,
                box_visible=True,
                meanline_visible=True,
                showlegend=False,
                line_color='black',
                fillcolor=problem["Color"][group],
                marker = dict(color=problem["Color"][group], line=dict(color='black',width=0.3), opacity=0.8),
                points='all',
                bandwidth = np.ptp(vals) / dist_num_bins,
                opacity=0.8
            )
        )
    add_layout(fig, "", "Methylation", f"{cpg} ({gene})<br>p-value: {pval:0.2e}<br>log(Fold Change): {log_fc:0.2e}")
    fig.update_layout(title_xref='paper', title={'y': 0.95})
    fig.update_layout(legend_font_size=25)
    fig.update_layout(legend={'itemsizing': 'constant'})
    fig.update_xaxes(tickfont_size=25)
    fig.update_layout(
        violingap=0.35,
        violingroupgap=0.35,
        width=850,
        height=615,
        margin=go.layout.Margin(
            l=150,
            r=50,
            b=75,
            t=115,
            pad=0,
        )
    )
    save_figure(fig, f"{path_save}/{path_local}/examples/{cpg_id}_{cpg}")